In [1]:
%pip install plotly
%pip install pandas
%pip install ta
import ta
import plotly.graph_objects as go
import plotly.subplots as sp
import requests
import json
import datetime
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import time

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [62]:
# Kraken data
pair = "XBTUSDT"
interval = 60

url = f"https://api.kraken.com/0/public/OHLC?pair={pair}&interval={interval}"
response = requests.get(url)
data = json.loads(response.text)

ohlcv_data = data['result'][pair]
times = [datetime.datetime.fromtimestamp(int(entry[0])) for entry in ohlcv_data]
opens = [entry[1] for entry in ohlcv_data]
highs = [entry[2] for entry in ohlcv_data]
lows = [entry[3] for entry in ohlcv_data]
closes = [entry[4] for entry in ohlcv_data]

df = pd.DataFrame(list(zip(times, opens, highs, lows, closes)), columns=['time', 'open', 'high', 'low', 'close'])
df['time'] = pd.to_datetime(df['time'])
df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].apply(pd.to_numeric)

df['atr'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=14)
df.drop(df.index[:14], inplace=True)
df.reset_index(drop=True, inplace=True)

In [63]:
pd.set_option('display.max_rows', 100)

In [196]:
def calculate_resistance(df):
    resistance_lines = []
    i = 0
    while i < len(df) - 10:  # -10 to leave space for checking the next 10 candles
        current_candle = df.iloc[i]
        if current_candle['close'] > current_candle['open']:  # Check if the candle is green
            upper_range = current_candle['close'] + (0.5 * current_candle['atr'])
            lower_range = current_candle['close'] - (0.5 * current_candle['atr'])
            bottom_limit = current_candle['close'] - (5 * current_candle['atr'])
            second_touch = False
            next_10_candles = df.iloc[i+1:i+11]
            
            # If any of the next 10 candles' closing prices exceed the upper limit, we skip to the next current candle
            if any(next_10_candles['close'] > upper_range):
                i += 1
                continue
                
            for j in range(i+11, len(df)):  # Skip next 10 candles and then start looking for touch points
                next_candle = df.iloc[j]
                
                # Check if next_candle close is within the touch point range
                if lower_range <= next_candle['close'] <= upper_range:
                    second_touch = True
                
                # Check if next_candle close is outside the allowed range, even if there's no second touch yet
                if next_candle['close'] > upper_range or next_candle['close'] < bottom_limit:
                    if second_touch:  # If we had a second touch, we append the line
                        resistance_lines.append({'start_time': current_candle['time'], 'end_time': next_candle['time'], 'price': current_candle['close']})
                    second_touch = False  # Reset second_touch to False
                    i = j + 1  # Start analyzing from the next candle
                    break
            else:
                i += 1  # If we finished the loop without hitting a break, increment i to check the next candle
        else:
            i += 1  # If the current candle is not green, check the next one
    return pd.DataFrame(resistance_lines)

resistance_df = calculate_resistance(df)

In [197]:
resistance_df['start_time'] = pd.to_datetime(resistance_df['start_time'])
resistance_df['end_time'] = pd.to_datetime(resistance_df['end_time'])
resistance_df['length'] = (resistance_df['end_time'] - resistance_df['start_time']).dt.total_seconds() / (60 * 60)
resistance_df['length'] = resistance_df['length'].astype(int)
resistance_df

,start_time,end_time,price,length
0,2023-05-02 14:00:00,2023-05-03 17:00:00,28720.2,27
1,2023-05-03 18:00:00,2023-05-04 21:00:00,29146.5,27
2,2023-05-06 14:00:00,2023-05-07 10:00:00,28923.8,20
3,2023-05-12 05:00:00,2023-05-12 17:00:00,26390.1,12
4,2023-05-12 19:00:00,2023-05-13 16:00:00,26788.7,21
5,2023-05-13 18:00:00,2023-05-14 10:00:00,26891.0,16
6,2023-05-15 02:00:00,2023-05-17 04:00:00,27476.0,50
7,2023-05-17 15:00:00,2023-05-18 13:00:00,27363.3,22
8,2023-05-18 17:00:00,2023-05-20 13:00:00,26927.2,44
9,2023-05-22 09:00:00,2023-05-22 22:00:00,27013.7,13


In [198]:
fig = sp.make_subplots(specs=[[{"secondary_y": True}]])

candlestick_trace = go.Candlestick(x=df['time'], open=df['open'], high=df['high'], low=df['low'], close=df['close'])
atr_trace = go.Scatter(x=df['time'], y=df['atr'], mode='lines', name='ATR', line=dict(color='purple'), opacity=0.2)

fig.add_trace(candlestick_trace, secondary_y=False)
fig.add_trace(atr_trace, secondary_y=True)

fig.update_layout(title='BTC/USDT', xaxis_title='Time')
fig.update_yaxes(title_text='Price', secondary_y=False, tickformat="$,.2f", autorange=True, fixedrange=False)
fig.update_yaxes(title_text='ATR', secondary_y=True)

for _, row in resistance_df.iterrows():
    start_time = row['start_time']
    end_time = row['end_time']
    price = row['price']
    
    fig.add_shape(type="line", x0=start_time, y0=price, x1=end_time, y1=price, line=dict(color='blue', width=3), xref='x', yref='y')

fig.show()

In [ ]:
# BTC data (10,000 points)

"""df = pd.read_csv('btc_data.csv')

df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
df.columns = df.columns.str.lower()
df.rename(columns={'timestamp': 'date'}, inplace=True)
df.drop(['volume_(btc)', 'weighted_price', 'volume_(currency)'], axis=1, inplace=True)
df.set_index('date', inplace=True)
df.fillna(method='ffill', inplace=True)

df = df.iloc[700000:]
df = df.resample('H').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'})
df = df.iloc[59399:]
df.reset_index(inplace=True)

fig = go.Figure(data=[go.Candlestick(x=df['date'], open=df['open'], high=df['high'], low=df['low'], close=df['close'])])
fig.update_layout(title='BTC Candlestick Chart', xaxis_title='Date', yaxis_title='Price')
fig.show()"""